# 🤗 Fine-Tune BERT for Phishing Email Detection
Using the [TanvirAhammed/phishing_email_dataset](https://huggingface.co/datasets/TanvirAhammed/phishing_email_dataset) from Hugging Face.

**Steps:**
1. Load the dataset
2. Preprocess and tokenize
3. Fine-tune BERT
4. Evaluate the model

In [ ]:
# Install Hugging Face libraries
!pip install transformers datasets accelerate -q

In [ ]:
# Step 1: Load the dataset
from datasets import load_dataset
dataset = load_dataset("TanvirAhammed/phishing_email_dataset")
dataset

In [ ]:
# Step 2: Tokenize using BERT tokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(example):
    return tokenizer(example['text'], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
# Step 3: Prepare the training pipeline
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    tokenizer=tokenizer,
)

In [ ]:
# Step 4: Train the model
trainer.train()

In [ ]:
# Step 5: Evaluate the model
trainer.evaluate()